- Database: Covid-fake
- Function: cleaning
- Desp: NA

# Necessary Imports...

In [1]:
# Start writing code here...
import pandas as pd
import xgboost
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import neptune
from neptunecontrib.monitoring.xgboost import neptune_callback
with open("token", "r") as f:
    token = f.read()
neptune.init(project_qualified_name='tathagataraha/contro-base',
             api_token=token,
             )

Project(tathagataraha/contro-base)

In [2]:
params = {
    'rnd' : 42,
    'tsize' : 0.20,
    'vsize' : 0.125,
    'label' : 'is_cont'
}


# Read test and training data

In [6]:
trains = pd.read_csv('data/dataset.csv')
# test = pd.read_csv('data/public_dev.csv')
train,test = train_test_split(trains,test_size=params['tsize'],random_state=params['rnd'])


In [7]:
train.shape

(2167, 8)

In [8]:
train.head(10)

,time,id,text,user_id,user_name,no_of_retweets,no_of_likes,is_cont
445,2020-04-23 23:19:24,1253463512464216068,Pandemic potentially a 'death sentence' for ma...,16589206,wikileaks,215,286,1
986,2020-11-06 06:35:51,1324601373296676866,Nope. You’re being deliberately dishonest.\n\n...,23022687,tedcruz,6385,22782,1
2396,2021-02-22 02:20:25,1363674979464798213,Stars showing out at the half on ESPN.. @Brook...,19923144,NBA,75,783,0
1593,2021-01-11 04:43:57,1348490811248439296,The 2022 PGA Championship will not be played a...,428333,cnnbrk,1281,13972,1
748,2021-01-12 22:22:37,1349119620939210754,That would make too much sense. https://t.co/t...,23022687,tedcruz,3919,27139,1
2553,2021-02-19 23:00:00,1362899765940211715,▪️ Zion coming off career-high tying 36 PTS as...,19923144,NBA,50,414,0
2303,2021-02-23 22:00:01,1364334222677336070,Where else can you see LUKA MAGIC at any momen...,19923144,NBA,120,1030,0
1870,2020-04-20 19:35:09,1252319916566462465,Chris had the great pleasure of performing wit...,18863815,coldplay,318,2302,0
1451,2021-01-20 19:47:01,1351979563023216644,President Biden and Vice President Harris are ...,428333,cnnbrk,512,7339,1
999,2020-11-03 06:10:41,1323507874379816960,nope. https://t.co/kNzYIBYxBc,23022687,tedcruz,2148,17720,1


# Label Encoding

In [6]:
train.head(10)

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
1467,1468,customer: i'd like to return this boomerang me...,1,2.30,1.0,0.00
5768,5769,"Keep your ears ready for next week, when The A...",0,NaN,NaN,0.50
5714,5715,[2 am] *5 year old sneaks into my room* 5: (wh...,1,3.00,0.0,0.00
1578,1579,Sex and food activate the same parts of the br...,0,NaN,NaN,0.10
6958,6959,"Gay or straight, No state should legally recog...",1,2.25,1.0,0.30
6861,6862,I got into a fight with my erection this morni...,1,2.45,1.0,1.05
2126,2127,Mammals evolved before flowering plants.,0,NaN,NaN,0.00
6202,6203,I know several jokes in sign language. I guara...,1,2.95,1.0,0.90
1450,1451,"A black guy broke into my house last night, lu...",1,1.80,1.0,3.40
5067,5068,"""All any one needs is someone to step in and l...",0,NaN,NaN,0.00


# Cleaning training and test data

In [11]:
train = train.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
train.text = train.text.apply(clean_text)
train.text = train.text.str.replace('\d+', '')

<ipython-input-11-2ebb5082d0c8>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  train.text = train.text.str.replace('\d+', '')


### Preparing Test Data

In [12]:
test = train.sample(1000)
test = test.reset_index(drop=True)
test.text = test.text.apply(clean_text)
test.text = test.text.str.replace('\d+', '')

<ipython-input-12-211f84607c0a>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test.text = test.text.str.replace('\d+', '')


In [13]:
train.text

0       pandemic potentially a death sentence for many...
1       nope youre being deliberately dishonestsomeone...
2       stars showing out at the half on espn brooklyn...
3       the pga championship will not be played at tru...
4       that would make too much sense https tco tgjiv...
                              ...                        
2162    the bands a head full of dreams documentary fi...
2163    the international atomic energy agency has str...
2164    texas customers could experience outages in ho...
2165    the biden administration announces it will soo...
2166                            uh oh https tco hkyygaokn
Name: text, Length: 2167, dtype: object

# Base Line Model Used
## 1. Naive Bayes
## 2. Linear Classifier
## 3. Bagging
## 4. Boosting
## 5. SVM

### Building Model

In [14]:
def train_model(classifier, feature_vector_train, label,  feature_vector_valid, valid_y,test_data , test_label ,is_neural_net=False, xgb = False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #print("In Validation Data",metrics.accuracy_score(predictions, valid_y))
    #applying in test data
    predictions_test = classifier.predict(test_data)
    
    if is_neural_net:
        predictions_test = predictions_test.argmax(axis=-1)
    print("f1 score: ",f1_score(test_label,predictions_test))
        
    return classifier, f1_score(test_label,predictions_test), metrics.accuracy_score(test_label,predictions_test)
    

### 1.Splitting the Data into Train and validation

In [15]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(train['text'], train[params['label']], test_size = params['vsize'], random_state = params['rnd'])

### 2. Applying WordLevel tf-idf and bi-gram tf-idf

In [16]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
test_tfidf   =  tfidf_vect.transform(test['text'])

# ngram level tf-idf (bigram in this case)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
test_tfidf_ngram   =  tfidf_vect.transform(test['text'])

In [17]:
xtest_tf = tfidf_vect.transform(test.text)
xtest_tf.shape

(1000, 5000)

In [18]:
import pickle
with open('chk.pkl', 'wb+') as f:
    pickle.dump(tfidf_vect, f)

#  Naive Bayes Model 

In [20]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y, test_tfidf, test[params['label']])
print ("NB, WordLevel TF-IDF: ", accuracy)
params['classifier'] = 'nb'
params['tf'] = 'unigram'
neptune.create_experiment(params = params)
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])
# Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y ,test_tfidf_ngram, test['is_humor'])
# print ("NB, Bi-Gram Vectors: ", accuracy)
# params['tf'] = 'bi'
# neptune.create_experiment(params = params)
# neptune.log_metric('accuracy', accuracy[2])
# neptune.log_metric('f1', accuracy[1])

f1 score:  0.9851239669421487
NB, WordLevel TF-IDF:  (MultinomialNB(), 0.9851239669421487, 0.982)
https://ui.neptune.ai/tathagataraha/contro-base/e/CON-1


# Linear Classifier

In [21]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y, test_tfidf, test[params['label']])
print("LR, WordLevel TF-IDF: ", accuracy)
params['classifier'] = 'linear'
params['tf'] = 'unigram'
neptune.create_experiment(params = params)
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(),  xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y ,test_tfidf_ngram, test[params['label']])
print("LR, Bi-Gram Vectors: ", accuracy)
params['tf'] = 'bi'
neptune.create_experiment(params = params)
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])

f1 score:  0.9844134536505331
LR, WordLevel TF-IDF:  (LogisticRegression(), 0.9844134536505331, 0.981)
https://ui.neptune.ai/tathagataraha/contro-base/e/CON-2
f1 score:  0.7485677912157862
LR, Bi-Gram Vectors:  (LogisticRegression(), 0.7485677912157862, 0.605)
https://ui.neptune.ai/tathagataraha/contro-base/e/CON-3


# Bagging Model

In [22]:
# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y, test_tfidf, test[params['label']])
print ("RF, WordLevel TF-IDF: ", accuracy)
params['classifier'] = 'bagging'
params['tf'] = 'unigram'
neptune.create_experiment(params = params)
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])
# RF on ngram Level TF IDF Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y ,test_tfidf_ngram, test['is_humor'])
# print ("RF, Bi-gram TF-IDF: ", accuracy)
# params['tf'] = 'bi'
# neptune.create_experiment(params = params)
# neptune.log_metric('accuracy', accuracy[2])
# neptune.log_metric('f1', accuracy[1])

f1 score:  0.99009900990099
RF, WordLevel TF-IDF:  (RandomForestClassifier(), 0.99009900990099, 0.988)
https://ui.neptune.ai/tathagataraha/contro-base/e/CON-4


In [33]:
preds = accuracy[0].predict(xtest_tf)
preds
df = pd.DataFrame()
df['id'] = list(range(8001,9001))
df['is_humor'] = list(preds)
df.to_csv('preds1.csv', index=False)

# Boosting Model

In [23]:

# Extereme Gradient Boosting on Word Level TF IDF Vectors
neptune.create_experiment(params = params)
accuracy = train_model(xgboost.XGBClassifier(callbacks=[neptune_callback()]), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(), valid_y, test_tfidf.tocsc(), test[params['label']])
print("Xgb, WordLevel TF-IDF: ", accuracy)
params['classifier'] = 'boosting'
params['tf'] = 'unigram'
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])
# Extereme Gradient Boosting on ngram Level TF IDF Vectors
neptune.create_experiment(params = params)
accuracy = train_model(xgboost.XGBClassifier(callbacks=[neptune_callback()]), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc(), valid_y, test_tfidf_ngram.tocsc(), test[params['label']])
print("Xgb, Bi-gram TF-IDF: ", accuracy)
params['tf'] = 'bi'
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])

https://ui.neptune.ai/tathagataraha/contro-base/e/CON-5
[13:16:54] WARNING: ../src/learner.cc:516: 
Parameters: { callbacks } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


f1 score:  0.9836065573770492
Xgb, WordLevel TF-IDF:  (XGBClassifier(base_score=0.5, booster='gbtree',
              callbacks=[<function neptune_callback.<locals>.callback at 0x153bb705df70>],
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lamb

# SVM Model

In [24]:

#SVM Model on Unigram TF-IDF
accuracy = train_model(svm.SVC(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(), valid_y, test_tfidf.tocsc(), test[params['label']])
print("SVM, WordLevel TF-IDF: ", accuracy)
params['classifier'] = 'svm'
params['tf'] = 'unigram'
neptune.create_experiment(params = params)
neptune.log_metric('accuracy', accuracy[2])
neptune.log_metric('f1', accuracy[1])
# with open('/scratch/tathagataraha/covid-svm-uni.pkl', 'wb+') as f:
#     pickle.dump(model, f)
# SVM Model on Bigram TF-IDF
# model, accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc(), valid_y, test_tfidf_ngram.tocsc(), test['label'])
# print("SVM, Bi-gram TF-IDF: ", accuracy)

f1 score:  0.9917355371900827
SVM, WordLevel TF-IDF:  (SVC(), 0.9917355371900827, 0.99)
https://ui.neptune.ai/tathagataraha/contro-base/e/CON-7
